# Quantity

Brainunit generates standard names for units, combining the unit name (e.g. “siemens”) with a prefixes (e.g. “m”), and also generates squared and cubed versions by appending a number. For example, the units “msiemens”, “siemens2”, “usiemens3” are all predefined. You can import these units from the package `brianunit` – accordingly, an `from brainunit import *` will result in everything being imported.

We recommend importing only the units you need, to have a cleaner namespace. For example, `import brainunit as bu` and then using `bu.msiemens` instead of `msiemens`.

In [1]:
import brainunit as bu

In the underlying design, Quantity consists of two attributes (mantissa and unit), and the specific value it represents is calculated by the formula below:

value = `mantissa` * (`unit.base` ^ (`unit.scale`)), 

where `unit.base` is the base for this unit (as the base of the exponent) and `unit.scale` is the scale for this unit (as the integer exponent of base)

You can generate a physical quantity by multiplying a scalar or ndarray with its physical unit:

In [2]:
tau = 20 * bu.ms
tau

20 * msecond

In [3]:
rates = [10, 20, 30] * bu.Hz
rates

ArrayImpl([10, 20, 30]) * hertz

In [4]:
rates = [[10, 20, 30], [20, 30, 40]] * bu.Hz
rates

ArrayImpl([[10, 20, 30],
           [20, 30, 40]]) * hertz

Brainunit will check the consistency of operations on units and raise an error for dimensionality mismatches:

In [5]:
try:
    tau += 1  # ms? second?
except Exception as e:
    print(e)

Cannot calculate ... += 1, units do not match (units are ms and Unit(1.0)).


In [6]:
try:
    3 * bu.kgram + 3 * bu.amp
except Exception as e:
    print(e)

Cannot calculate 3 + 3, units do not match (units are kg and A).


## Creating Quantity Instances

Creating a Quantity object can be accomplished in several ways, categorized based on the type of input used. Here, we present the methods grouped by their input types and characteristics for better clarity.

In [7]:
import jax.numpy as jnp
import brainstate as bst
bst.environ.set(precision=64) # we recommend using 64-bit precision for better numerical stability

### Scalar and Array Multiplication
- Multiplying a Scalar with a Unit

In [8]:
5 * bu.ms

5 * msecond

- Multiplying a Jax nunmpy value type with a Unit:

In [9]:
jnp.float64(5) * bu.ms

5. * msecond

- Multiplying a Jax numpy array with a Unit:

In [10]:
jnp.array([1, 2, 3]) * bu.ms

ArrayImpl([1, 2, 3], dtype=int64) * msecond

- Multiplying a List with a Unit:

In [11]:
[1, 2, 3] * bu.ms

ArrayImpl([1, 2, 3], dtype=int64) * msecond

### Direct Quantity Creation

- Creating a Quantity Directly with a Value

In [12]:
bu.Quantity(5)

Quantity(5)

- Creating a Quantity Directly with a Value and Unit

In [13]:
bu.Quantity(5, unit=bu.ms)

5 * msecond

- Creating a Quantity with a Jax numpy Array of Values and a Unit

In [14]:
bu.Quantity(jnp.array([1, 2, 3]), unit=bu.ms)

ArrayImpl([1, 2, 3], dtype=int64) * msecond

- Creating a Quantity with a List of Values and a Unit

In [15]:
bu.Quantity([1, 2, 3], unit=bu.ms)

ArrayImpl([1, 2, 3], dtype=int64) * msecond

- Creating a Quantity with a List of Quantities

In [16]:
bu.Quantity([500 * bu.ms, 1 * bu.second])

ArrayImpl([ 500., 1000.]) * msecond

- Using the with_units Method

In [19]:
bu.Quantity.with_unit(jnp.array([0.5, 1]), unit=bu.second)

ArrayImpl([0.5, 1. ]) * second

### Unitless Quantity
Quantities can be unitless, which means they have no units. If there is no unit provided, the quantity is assumed to be unitless. The following are examples of creating unitless quantities:

In [20]:
bu.Quantity([1, 2, 3])

Quantity(ArrayImpl([1, 2, 3], dtype=int64))

In [21]:
bu.Quantity(jnp.array([1, 2, 3]))

Quantity(ArrayImpl([1, 2, 3], dtype=int64))

In [22]:
bu.Quantity([])

Quantity(ArrayImpl([], dtype=float64))

### Illegal Quantity Creation
The following are examples of illegal quantity creation:

In [23]:
try:
    bu.Quantity([500 * bu.ms, 1])
except Exception as e:
    print(e)

All elements must have the same units, but got [msecond, None]


In [24]:
try:
    bu.Quantity(["some", "nonsense"])
except Exception as e:
    print(e)

Value 'some' with dtype <U4 is not a valid JAX array type. Only arrays of numeric types are supported by JAX.


In [25]:
try:
    bu.Quantity([500 * bu.ms, 1 * bu.volt])
except Exception as e:
    print(e)

All elements must have the same units, but got [msecond, volt]


### Creating Functions
You can create functions that return quantities. The following are examples of creating functions that return quantities:

In [28]:
import brainunit.math as bm

#### `brainunit.math.array` & `brainunit.math.asarray`
Convert the input to a quantity or array.

  If unit is provided, the input will be checked whether it has the same unit as the provided unit.
  (If they have same unit but different scale, the input will be converted to the provided unit.)
  If unit is not provided, the input will be converted to an array.


In [29]:
bm.asarray([1, 2, 3])                       # return a jax.Array

Array([1, 2, 3], dtype=int64)

In [35]:
# check if the input has the same unit as the provided unit
bm.asarray([1, 2, 3] * bu.second, unit=bu.second)    # return a Quantity

ArrayImpl([1, 2, 3], dtype=int64) * second

In [37]:
# Same unit, but different scale
bm.asarray([1, 2, 3] * bu.msecond, unit=bu.second)    # return a Quantity

ArrayImpl([0.001, 0.002, 0.003]) * second

In [38]:
# fails because the input has a different unit
try:
    bm.asarray([1 * bu.second, 2 * bu.second], unit=bu.ampere)
except Exception as e:
    print(e)

Cannot convert to a unit with different dimensions. (units are s and A).


#### More Functions
Other functions that can be used to create quantities are:
- `brainunit.math.arange`
- `brainunit.math.array_spilt`
- `brainunit.math.linespace`
- `brainunit.math.logspace`
- `brainunit.math.meshgrid`
- `brainunit.math.vandermode`
- Can use with Quantity
  - `brainunit.math.full`
  - `brainunit.math.empty`
  - `brainunit.math.ones`
  - `brainunit.math.zeros`
  - `brainunit.math.full_like`
  - `brainunit.math.empty_like`
  - `brainunit.math.ones_like`
  - `brainunit.math.zeros_like`
  - `brainunit.math.fill_diagonal`
- Can use with unit keyword
  - `brainunit.math.eye`
  - `brainunit.math.identity`
  - `brainunit.math.tri`
  - `brainunit.math.diag`
  - `brainunit.math.tril`
  - `brainunit.math.triu`

See the [Array Creation Documentation](https://brainunit.readthedocs.io/en/latest/mathematical_functions/array_creation.html) for more information.

## Converting to Different Units

You can convert a quantity to a different unit using the `to_decimal` method. The following are examples of converting quantities to different units:

In [40]:
q = (1, 2, 3) * bu.second
q

ArrayImpl([1, 2, 3], dtype=int64) * second

In [41]:
q.to_decimal(bu.msecond)

Array([1000., 2000., 3000.], dtype=float64, weak_type=True)

Brainunit will check the consistency of operations on units and raise an error for dimensionality mismatches:

In [42]:
try:
    q.to_decimal(bu.ampere)
except Exception as e:
    print(e)

Cannot convert to the decimal number using a unit with different dimensions. The quantity has the unit s, but the given unit is A


## Attributes of a Quantity

The important attributes of a `Quantity` object are:
- `mantissa`: the mantissa of the quantity
- `unit`: the unit of the quantity
- `dim`: the dimension of the unit of the quantity
- `ndim`: the number of dimensions of quantity's value
- `shape`: the shape of the quantity's value
- `size`: the size of the quantity's value
- `dtype`: the dtype of the quantity's value

### An example

In [48]:
rates = [[10., 20., 30.], [20., 30., 40.]] * bu.Hz
rates

ArrayImpl([[10., 20., 30.],
           [20., 30., 40.]]) * hertz

In [49]:
rates.mantissa

Array([[10., 20., 30.],
       [20., 30., 40.]], dtype=float64)

In [50]:
rates.dim

second ** -1

In [51]:
rates.ndim, rates.shape, rates.size, rates.dtype

(2, (2, 3), 6, dtype('float64'))

## Arithmetic Functions

Like Numpy and Jax numpy, arithmetic operators on arrays apply elementwise.

In [28]:
a = [20, 30, 40, 50] * bu.mV
b = jnp.arange(4) * bu.mV
b

ArrayImpl([0., 1., 2., 3.]) * mvolt

### Addition and Subtraction

Addition and subtraction of quantities need to have the same units and keep the units in the result.

In [29]:
c = a - b
c

ArrayImpl([20., 29., 38., 47.]) * mvolt

In [30]:
c + b

ArrayImpl([20., 30., 40., 50.]) * mvolt

### Multiplication and Division

Multiplication and division of quantities multiply and divide the values and add and subtract the dimensions of the units.

In [31]:
A = jnp.array([[1, 2], [3, 4]]) * bu.mV
B = jnp.array([[5, 6], [7, 8]]) * bu.mV

A, B

(ArrayImpl([[1., 2.],
            [3., 4.]]) * mvolt,
 ArrayImpl([[5., 6.],
            [7., 8.]]) * mvolt)

In [32]:
A * B # element-wise multiplication

ArrayImpl([[ 5., 12.],
           [21., 32.]]) * mvolt2

In [33]:
A @ B # matrix multiplication

ArrayImpl([[19., 22.],
           [43., 50.]]) * mvolt2

In [34]:
A.dot(B) # matrix multiplication

ArrayImpl([[19., 22.],
           [43., 50.]]) * mvolt2

In [35]:
A / 2 # divide by a scalar

ArrayImpl([[0.5, 1. ],
           [1.5, 2. ]]) * mvolt

if the unit of result is unitless, the unit is removed and returned as jax.Array

In [38]:
A / (2 * bu.mV) # divide by a quantity, return jax array

Array([[0.5, 1. ],
       [1.5, 2. ]], dtype=float64, weak_type=True)

In [40]:
A / (2 * bu.mA) # divide by a quantity, return quantity

ArrayImpl([[0.5, 1. ],
           [1.5, 2. ]]) * ohm

### Power

The power operator raises the value of the quantity to the power of the scalar, and multiplies the unit by the scalar.

In [48]:
A

ArrayImpl([[1., 2.],
           [3., 4.]]) * mvolt

In [49]:
A ** 2 # element-wise power

ArrayImpl([[ 1.,  4.],
           [ 9., 16.]]) * mvolt2

## Built-in Functions

Brainunit provides a number of built-in functions in `Quantity` class to perform operations on quantities. These functions are:
- unary operations
    - positive(+)
    - negative(-)
    - absolute(abs)
    - invert(~)
- logical operations
    - all
    - any
- shape operations
    - reshape
    - resize
    - squeeze
    - unsqueeze
    - spilt
    - swapaxes
    - transpose
    - ravel
    - take
    - repeat
    - diagonal
    - trace
- mathematical functions
    - nonzero
    - argmax
    - argmin
    - argsort
    - var
    - round
    - std
    - sum
    - cumsum
    - cumprod
    - max
    - mean
    - min
    - ptp
    - clip
    - conj
    - dot
    - fill
    - item
    - prod
    - clamp
    - sort

For more details on these functions, refer to the [documentation](https://braincore.readthedocs.io/en/latest/braincore/apis/generated/brainunit.Quantity.html).

## Indexing, Slicing and Iterating

One-dimensional Quantity can be indexed, sliced and iterated over, much like lists and other Python sequences.

In [52]:
a = jnp.arange(10) ** 3 * bu.mV
a

ArrayImpl([  0,   1,   8,  27,  64, 125, 216, 343, 512, 729], dtype=int64) * mvolt

In [53]:
a[2]

8 * mvolt

In [54]:
a[2:5]

ArrayImpl([ 8, 27, 64], dtype=int64) * mvolt

Only same dimension Quantity can be set to a slice of a Quantity.

In [55]:
# equivalent to a[0:6:2] = 1000;
# from start to position 6, exclusive, set every 2nd element to 1000
a[:6:2] = 1000 * bu.mV
a

ArrayImpl([1000,    1, 1000,   27, 1000,  125,  216,  343,  512,  729],
          dtype=int64) * mvolt

In [56]:
a[::-1] # reversed a

ArrayImpl([ 729,  512,  343,  216,  125, 1000,   27, 1000,    1, 1000],
          dtype=int64) * mvolt

In [57]:
for i in a:
    print(i**(1 / 3.))

10. mV^0.3333333333333333
1. mV^0.3333333333333333
10. mV^0.3333333333333333
3. mV^0.3333333333333333
10. mV^0.3333333333333333
5. mV^0.3333333333333333
6. mV^0.3333333333333333
7. mV^0.3333333333333333
8. mV^0.3333333333333333
9. mV^0.3333333333333333


Multidimensional Quantity can have one index per axis. These indices are given in a tuple separated by commas:

In [58]:
def f(x, y):
    return 10 * x + y
b = jnp.fromfunction(f, (5, 4), dtype=jnp.int32) * bu.mV
b

ArrayImpl([[ 0,  1,  2,  3],
           [10, 11, 12, 13],
           [20, 21, 22, 23],
           [30, 31, 32, 33],
           [40, 41, 42, 43]]) * mvolt

In [59]:
b[2, 3]

23 * mvolt

In [60]:
b[0:5, 1]  # each row in the second column of b

ArrayImpl([ 1, 11, 21, 31, 41]) * mvolt

In [61]:
b[:, 1]  # equivalent to the previous example

ArrayImpl([ 1, 11, 21, 31, 41]) * mvolt

In [62]:
b[1:3, :]  # each column in the second and third row of b

ArrayImpl([[10, 11, 12, 13],
           [20, 21, 22, 23]]) * mvolt

When fewer indices are provided than the number of axes, the missing indices are considered complete slices:

In [63]:
b[-1]

ArrayImpl([40, 41, 42, 43]) * mvolt

The expression within brackets in b[i] is treated as an i followed by as many instances of : as needed to represent the remaining axes. NumPy also allows you to write this using dots as b[i, ...].

The dots (...) represent as many colons as needed to produce a complete indexing tuple. For example, if x is a Quantity with 5 axes, then
- x[1, 2, ...] is equivalent to x[1, 2, :, :, :],
- x[..., 3] to x[:, :, :, :, 3] and
- x[4, ..., 5, :] to x[4, :, :, 5, :].

In [64]:
c = jnp.array([[[0, 1, 2], [10, 12, 13]], [[100, 101, 102], [110, 112, 113]]]) * bu.mV # a 3D array (two stacked 2D arrays)
c.shape

(2, 2, 3)

In [65]:
c[1, ...] # same as c[1, :, :] or c[1]

ArrayImpl([[100, 101, 102],
           [110, 112, 113]], dtype=int64) * mvolt

In [66]:
c[..., 2] # same as c[:, :, 2]

ArrayImpl([[  2,  13],
           [102, 113]], dtype=int64) * mvolt

Iterating over multidimensional Quantity is done with respect to the first axis:

In [67]:
for row in b:
    print(row)

[0 1 2 3] mV
[10 11 12 13] mV
[20 21 22 23] mV
[30 31 32 33] mV
[40 41 42 43] mV


## Operating on Subsets

`.at` method can be used to operate on a subset of the Quantity. The following are examples of operating on subsets of a Quantity:

In [73]:
q = jnp.arange(5.0) * bu.mV
q

ArrayImpl([0., 1., 2., 3., 4.]) * mvolt

In [75]:
q.at[2].add(10 * bu.mV)

ArrayImpl([ 0.,  1., 12.,  3.,  4.]) * mvolt

In [74]:
q.at[10].add(10 * bu.mV)  # out-of-bounds indices are ignored

ArrayImpl([0., 1., 2., 3., 4.]) * mvolt

In [78]:
q.at[20].add(10 * bu.mV, mode='clip') # out-of-bounds indices are clipped

ArrayImpl([ 0.,  1.,  2.,  3., 14.]) * mvolt

In [79]:
q.at[2].get()

2. * mvolt

In [80]:
q.at[20].get()  # out-of-bounds indices clipped

4. * mvolt

In [81]:
q.at[20].get(mode='fill')  # out-of-bounds indices filled with NaN

nan * mvolt

Brainunit will check the consistency of operations on units and raise an error for dimensionality mismatches:

In [82]:
try:
    q.at[2].add(10)
except Exception as e:
    print(e)

Cannot convert to a unit with different dimensions. (units are Unit(1.0) and mV).


Brainunit also allows customized fill values for the `at` method:

In [83]:
q.at[20].get(mode='fill', fill_value=-1 * bu.mV)  # custom fill value

-1. * mvolt

In [84]:
try:
    q.at[20].get(mode='fill', fill_value=-1)
except Exception as e:
    print(e)

Cannot convert to a unit with different dimensions. (units are Unit(1.0) and mV).
